In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

import shapely
from shapely.geometry import LineString, MultiPoint
from shapely.ops import split, substring

import zlib
import datetime as dt

/home/edasmalchi/miniconda3/lib/python3.9/site-packages/google/auth/_default.py:71: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### High Quality Transit Areas Relevant Statutes

[PRC 21155](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21155.&lawCode=PRC)
* Major transit stop definition: _A major transit stop is as defined in Section 21064.3, except that, for purposes of this section, it also includes major transit stops that are included in the applicable regional transportation plan_
* High-quality transit corridor definition: _For purposes of this section, a high-quality transit corridor means a corridor with fixed route bus service with service intervals no longer than 15 minutes during peak commute hours._
    * Unable to locate definition of "peak commute hours"

[PRC 21064.3](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21064.3.&lawCode=PRC)
* _Major transit stop means a site containing any of the following:
(a) An existing rail or bus rapid transit station.
(b) A ferry terminal served by either a bus or rail transit service.
(c) The intersection of two or more major bus routes with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods._
    * "Intersection" may not be sufficiently well-defined for this analysis

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

In [2]:
##definition of "peak" pending
am_peak = range(6, 9)
pm_peak = range(16, 19)

In [3]:
am_commute_hours = list(am_peak)
pm_commute_hours = list(pm_peak)
commute_hours = am_commute_hours + pm_commute_hours

In [4]:
commute_hours

[6, 7, 8, 16, 17, 18]

In [5]:
def create_segments(geometry):
    '''Splits a Shapely LineString into smaller LineStrings. If a MultiLineString passed,
    splits each LineString in that collection. 
    '''
    
    lines = []
    segment_distance_meters = 1250
    geometry = geometry.iloc[0]
    if hasattr(geometry, 'geoms'): ##check if MultiLineString
        linestrings = geometry.geoms
    else:
        linestrings = [geometry]
    for linestring in linestrings:
        for i in range(0, int(linestring.length), segment_distance_meters):
            lines.append(substring(linestring, i, i+segment_distance_meters))
    return lines

In [6]:
def find_stop_with_high_trip_count(segment, stops, stop_times, rank, calculated_stops):
    '''Given a shape segment, finds the stop serving the most (or other rank) trips within that segment.
    Adds that stop's stop_id to segment data (a row).
    '''
    
    stops_in_seg = gpd.clip(stops, segment.geometry)
    if stops_in_seg.size == 0:
        return segment
    
    stop_times_in_seg = stops_in_seg >> inner_join(_, stop_times, on = 'stop_id')
    trip_count_by_stop = stop_times_in_seg >> count(_.stop_id) >> arrange(-_.n) >> rename(n_trips = _.n)
    try:
        stop_id = trip_count_by_stop['stop_id'].iloc[rank - 1]

        if stop_id in list(calculated_stops):
            return segment
        segment['stop_id'] = stop_id
        segment['n_trips'] = trip_count_by_stop['n_trips'].iloc[rank - 1]
        return segment
    except IndexError:
        return segment

In [7]:
def get_operator_views(itp_id):
    '''Returns relevant views from the data warehouse for a single transit operator.
    '''
    shapes = tbl.gtfs_schedule.shapes() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
    shapes = gpd.GeoDataFrame(shapes, 
                              geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat),
                              crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    print('loaded shapes')
    if itp_id != 273:
        wednesday = (tbl.views.gtfs_schedule_fact_daily_service() 
                 >> filter(_.calitp_itp_id == int(itp_id)) >> collect())
        wednesday = wednesday >> arrange(-_.service_date)
        wednesday = wednesday[wednesday['service_date'].apply(lambda x: x.weekday() == 2)]
        wednesday = wednesday[wednesday['service_date'].apply(lambda x: x.month < 11)]
        wednesday = wednesday[wednesday['service_date'] == wednesday['service_date'].iloc[0]] ## pick most recent Wednesday from Oct or earlier
    
    bus_routes = (tbl.gtfs_schedule.routes()
                    >> filter(_.calitp_itp_id == int(itp_id))
                    >> filter(_.route_type.isin(['3', '11'])) ## bus and trolleybus
                    >> select(_.route_id) >> collect())
    print('loaded bus routes')
    
    if itp_id == 273:
        trips = (tbl.gtfs_schedule.trips()
         >> filter(_.calitp_itp_id == int(itp_id))
         >> filter(_.service_id.isin(['1'])) ## temporary hardcode for SacRT
         >> filter(_.route_id.isin(bus_routes.route_id))
         >> collect())
    else:
        trips = (tbl.gtfs_schedule.trips()
                 >> filter(_.calitp_itp_id == int(itp_id))
                 >> filter(_.service_id.isin(wednesday.service_id))
                 >> filter(_.route_id.isin(bus_routes.route_id))
                 >> collect())
    print('loaded trips')
    stop_times = (tbl.gtfs_schedule.stop_times()
                  >> filter(_.calitp_itp_id == int(itp_id))
                  >> collect())
    stop_times =  (stop_times >> filter(_.trip_id.isin(trips.trip_id))
                  >> select(-_.calitp_itp_id, -_.calitp_extracted_at))
    print('loaded stop times')

    stops = (tbl.gtfs_schedule.stops() 
             >> filter(_.calitp_itp_id == itp_id)
             >> select(_.stop_id, _.stop_lat, _.stop_lon)
             >> collect())
    stops = gpd.GeoDataFrame(stops,
                     geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                     crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    print('loaded stops')

    return shapes, trips, stop_times, stops

In [8]:
def fix_arrival_time(gtfs_timestring):
    '''Reformats a GTFS timestamp (which allows the hour to exceed 24 to mark service day continuity)
    to standard 24-hour time.
    '''
    split = gtfs_timestring.split(':')
    hour = int(split[0])
    if hour >= 24:
        split[0] = str(hour - 24)
        corrected = (':').join(split)
        return corrected.strip()
    else:
        return gtfs_timestring.strip()

In [9]:
def single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta):
    '''Starting with a single GTFS shape, split that shape into segments and determine if each segment qualifies
    as an HQTA. Existing segments within a shape are dropped for that shape, since their peak frequency and 
    HQTA status would have already been calculated for a previous shape.
    '''

    single_shape = (shapes
         >> filter(_.shape_id == shape_id)
         >> mutate(shape_pt_sequence = _.shape_pt_sequence.astype('int64'))
         >> arrange(_.shape_pt_sequence) ##arrange, then convert to line to preserve order...
        )
    
    route_line = LineString(list(single_shape['geometry']))
    single_line = single_shape[['calitp_itp_id', 'shape_id', 'calitp_extracted_at']].iloc[[0]] ##preserve info cols
    single_line['geometry'] = route_line
    single_line = gpd.GeoDataFrame(single_line, crs='EPSG:6414')
    
    if shape_id in debug_ids:
        print(f'***debug shape*** {shape_id}')
        debug_dict[f'{shape_id}_single_line'] = single_line
        debug_dict[f'{shape_id}_hqta'] = hqta

    calculated_stops = []
    if hqta.size != 0:
        already_calculated = hqta.dissolve(by='calitp_itp_id') ## get single polygon of HQTA calculation complete area
        single_line = single_line.overlay(already_calculated, how='difference') ## drop calculation complete area from current shape
        if single_line.size == 0:
            segments_with_max_stop = None
            print(f'no line for shape {shape_id}')
            return

    segmented = pd.DataFrame()
    for segment in create_segments(single_line.geometry):
        to_append = single_line.drop(columns=['geometry'])
        to_append['geometry'] = segment
        segmented = segmented.append(to_append)
        
    segmented = segmented.reset_index()
    segmented['segment_sequence'] = segmented.index.astype(str)
    segmented = segmented.astype({'calitp_itp_id': str})
    
    ## compute (hopefully unique) hash of segment id that can be used across routes/operators
    segmented['hqta_segment_id'] = segmented.apply(lambda x:
                zlib.crc32((x.calitp_itp_id + x.shape_id + x.segment_sequence).encode('utf-8')), axis=1)
    
    segmented.geometry = segmented.buffer(50) ##generous buffer for street/sidewalk width? Required to spatially find stops within each segment

    if shape_id in debug_ids:
        debug_dict[f'{shape_id}_segmented'] = segmented
        debug_dict[f'{shape_id}_stops'] = stops
        debug_dict[f'{shape_id}_route_ct_by_stop'] = route_count_by_stop
     
    segments_with_max_stop = segmented.apply(find_stop_with_high_trip_count, axis=1,
                                             args = (stops, stop_times, 1, calculated_stops))
    
    if not 'stop_id' in segments_with_max_stop.columns:
        segments_with_top2_stops = None
        print(f'no stops for shape {shape_id}')
        return ## no stops within segment

    max_stop_times = (stop_times 
                 >> select(_.stop_id, _.trip_id, _.departure_time)
                 >> inner_join(_, segments_with_max_stop, on = "stop_id")
                ) ## filter stop_times to the key stops in each segment
    max_stop_times = max_stop_times.dropna(subset=['departure_time'])
    max_stop_times = max_stop_times.drop_duplicates(subset=['trip_id','hqta_segment_id']) ## filter duplicates for top2 approach
    
    if shape_id in debug_ids:
        debug_dict[f'{shape_id}_max_stop0'] = max_stop_times
    
    max_stop_times['departure_time'] = max_stop_times['departure_time'].apply(fix_arrival_time) ## reformat GTFS time to a format datetime can ingest
    max_stop_times['departure_dt'] = max_stop_times['departure_time'].apply(lambda x:
                                                                    dt.datetime.strptime(x, '%H:%M:%S'))
    max_stop_times['departure_hour'] = max_stop_times['departure_dt'].apply(lambda x: x.hour)
    
    max_stop_times = max_stop_times >> filter(_.departure_hour.isin(commute_hours))
    
    if max_stop_times.size == 0:
        print(f'no commute hour trips for shape {shape_id}')
        return
    
    if shape_id in debug_ids:
        debug_dict[f'{shape_id}_max_stop'] = max_stop_times
        
    max_stop_times['am_peak'] = max_stop_times['departure_hour'].apply(lambda x: x in am_commute_hours)
    max_stop_times['pm_peak'] = ~max_stop_times['am_peak']
    
    max_stop_times.drop_duplicates(subset = ['stop_id', 'departure_dt'], inplace=True) ## drop duplicates if duplicitous service_ids sneak through
    max_stop_times.drop(columns=['departure_dt'], inplace=True)
    
    segment_peak_service = max_stop_times.groupby(['hqta_segment_id'])[['am_peak', 'pm_peak']].sum() ## count total trips at stop during each peak period
    
    ## convert to trips per hour
    segment_peak_service['am_peak'] = (segment_peak_service['am_peak'] / len(am_commute_hours)).round(1)
    segment_peak_service['pm_peak'] = (segment_peak_service['pm_peak'] / len(pm_commute_hours)).round(1)
    
    if shape_id in debug_ids:
        debug_dict[f'{shape_id}_segment_peak'] = segment_peak_service

    ## consider segment HQTA if stop within segment has at least 4 trips/hour in both peaks
    segment_peak_service['hq_transit_corr'] = segment_peak_service.apply(lambda x:
                              True if x.am_peak >= 4 and x.pm_peak >= 4 else False, axis=1)
    
    segment_peak_service = segment_peak_service.reset_index()
    segment_peak_service['last_seg_hqta'] = segment_peak_service['hq_transit_corr'].shift(1)
    segment_peak_service['next_seg_hqta'] = segment_peak_service['hq_transit_corr'].shift(-1)
    
    ## consider segment not HQTA if both the prior and next segements are not HQTAs
    segment_peak_service['hq_transit_corr'] = segment_peak_service.apply(
            lambda x: False if x.hq_transit_corr == False else x.last_seg_hqta or x.next_seg_hqta, axis = 1)
    segment_peak_service['hq_transit_corr'] = segment_peak_service['hq_transit_corr'].fillna(True)
    
    single_hqta = segments_with_max_stop >> inner_join(_, segment_peak_service, on = 'hqta_segment_id')
    single_hqta.drop(columns=['calitp_extracted_at', 'next_seg_hqta', 'last_seg_hqta'], inplace=True)
    
    if shape_id in debug_ids:
        debug_dict[f'{shape_id}_single_hqta'] = single_hqta
    
    return single_hqta

In [10]:
def single_operator_hqta(views):  
    
    global debug_dict
    debug_dict = {}
    
    shapes, trips, stop_times, stops = views
    
    
    distinct_routes = (trips
                   >> distinct(_.route_id, _.shape_id, _.direction_id, _keep_all = True)
                   >> select(_.calitp_itp_id, _.route_id, _.shape_id, _.direction_id, _.trip_id))
    
    route_count_by_stop = (stop_times
                         >> select(_.stop_id, _.trip_id)
                         >> inner_join(_, distinct_routes, on = "trip_id")
                         >> count(_.stop_id)
                         >> rename(n_routes = _.n)
                         >> arrange(-_.n_routes))    
    
    hqta = gpd.GeoDataFrame()
    ## start with shapes including the highest number of trips
    shapes_sorted = trips.groupby('shape_id').count().sort_values(by='trip_id', ascending=False).index
    shapes_sorted = pd.Series(shapes_sorted)
    total_shapes = len(shapes_sorted)
    print(f'there are {total_shapes} shapes total')
    for ix, shape_id in shapes_sorted.items():
        print(f'calculating for shape_id {shape_id}')
        if ix % 25 == 0:
            print(f'progress: {ix}/{total_shapes} shapes ({round(((ix/total_shapes)*100), 2)}%)')
        # try:
        result = single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop,
                                shape_id, hqta)
        hqta = hqta.append(result)
        # except:
        #     print(f'unable to calculate HQTA for shape_id {shape_id}')
        try:
            hqta = hqta.set_crs('EPSG:6414')
        except:
            continue
        
    return hqta

In [11]:
debug_ids = ['shp-6-69']

In [12]:
# ccb_views = get_operator_views(87)

In [13]:
# ccb_hqta = single_operator_hqta(ccb_views)

### Mapping

In [29]:
def map_hqta(hqta):
    
    x = hqta['geometry'].to_crs('EPSG:4326').iloc[0].centroid.x
    y = hqta['geometry'].to_crs('EPSG:4326').iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)
    geo_data_hq = GeoData(geo_dataframe = hqta[hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#08589e',
                                        'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'HQTA')

    geo_data_not_hq = GeoData(geo_dataframe = hqta[~hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#a8ddb5',
                                        'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'non-HQTA')

    m.add_layer(geo_data_hq)
    m.add_layer(geo_data_not_hq)
    m.add_control(LayersControl())
    
    return m

## Multiple Operators

In [15]:
itp_ids = tbl.gtfs_schedule.agency() >> distinct(_.calitp_itp_id) >> collect()
itp_ids = itp_ids.calitp_itp_id

In [16]:
def multiple_operator_hqta(existing=pd.DataFrame()):
    
    hqta = pd.DataFrame()    
    for _, itp_id in itp_ids.items():
        if not existing.empty:
            if str(itp_id) in existing['calitp_itp_id'].to_list():
                print(f'{itp_id} existing')
                # break
                continue
        if int(itp_id) == 200:
            continue ## skip MTC feed to use individual operator feeds
        try:
            print(f'attempting for operator {itp_id}')
            hqta = hqta.append(single_operator_hqta(get_operator_views(itp_id)))
        except:
            print(f'failed for operator {itp_id}')
        
    return hqta

In [17]:
# all_operators = multiple_operator_hqta()
# all_operators.to_parquet('./all_operators.parquet')

In [18]:
all_operators = gpd.read_parquet('./all_operators.parquet')

In [20]:
itp_ids = itp_ids.astype(str)

In [21]:
df = tbl.gtfs_schedule.agency() >> collect() >> distinct(_.calitp_itp_id, _keep_all = True)

In [22]:
not_ran = df[~df['calitp_itp_id'].isin(all_operators['calitp_itp_id'].astype('int64'))]

In [147]:
# with pd.option_context('display.max_rows', 100):
#     display(not_ran)

## Spatially Aggregate non-HQTAs

In [148]:
non_hqta = all_operators >> filter(-_.hq_transit_corr)

In [112]:
non_hqta_overlaid = non_hqta.overlay(nope, how='intersection')

In [113]:
non_hqta_deduplicated = non_hqta_overlaid[nope_overlaid['hqta_segment_id_1'] != non_hqta_overlaid['hqta_segment_id_2']]

In [140]:
def combine_segment_ids(row):
    both_segments = [row.hqta_segment_id_1, row.hqta_segment_id_2]
    both_segments.sort()
    return str(both_segments)

In [141]:
non_hqta_deduplicated['overlap_id'] = non_hqta_deduplicated.apply(combine_segment_ids, axis=1)

/home/edasmalchi/miniconda3/lib/python3.9/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [142]:
more_deduplicated = non_hqta_deduplicated.drop_duplicates(subset=['overlap_id'])

In [146]:
more_deduplicated.to_parquet('./data/bus/overlaps.parquet')

/tmp/ipykernel_14862/3646704021.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  more_deduplicated.to_parquet('./data/bus/overlaps.parquet')


In [150]:
more_deduplicated.columns

Index(['calitp_itp_id_1', 'hqta_segment_id_1', 'index_1', 'n_trips_1',
       'segment_sequence_1', 'shape_id_1', 'stop_id_1', 'am_peak_1',
       'pm_peak_1', 'hq_transit_corr_1', 'calitp_itp_id_2',
       'hqta_segment_id_2', 'index_2', 'n_trips_2', 'segment_sequence_2',
       'shape_id_2', 'stop_id_2', 'am_peak_2', 'pm_peak_2',
       'hq_transit_corr_2', 'geometry', 'overlap_id'],
      dtype='object')

In [160]:
def sum_overlaps(row):
    row['am_peak'] = row.am_peak_1 + row.am_peak_2
    row['pm_peak'] = row.pm_peak_1 + row.pm_peak_2
    if row.am_peak >= 4 and row.pm_peak >= 4:
        row['hq_transit_corr'] = True
    else:
        row['hq_transit_corr'] = False
    return row
    

In [161]:
overlaps_summed = more_deduplicated.apply(sum_overlaps, axis = 1) 

In [177]:
new_hq = overlaps_summed[overlaps_summed['hq_transit_corr']]

In [178]:
new_hq = new_hq[new_hq['am_peak'] != new_hq['am_peak_1']*2] ## drop likely duplicates; can rework above to do this better

In [179]:
new_hq = new_hq.drop(columns=['calitp_itp_id_2',
               'hqta_segment_id_2', 'index_2', 'n_trips_2', 'segment_sequence_2',
               'shape_id_2', 'stop_id_2', 'am_peak_2', 'pm_peak_2',
               'hq_transit_corr_2', 'index_1', 'index_2', 'n_trips_2'])

In [180]:
new_hq = new_hq.rename(columns = {'calitp_itp_id_1':'calitp_itp_id', 'hqta_segment_id_1':'hqta_segment_id', 'n_trips_1':'n_trips',
       'segment_sequence_1':'segment_sequence', 'shape_id_1':'shape_id', 'stop_id_1':'stop_id'})

In [183]:
new_hq

,calitp_itp_id,hqta_segment_id,n_trips,segment_sequence,shape_id,stop_id,am_peak_1,pm_peak_1,hq_transit_corr_1,geometry,overlap_id,am_peak,pm_peak,hq_transit_corr
1,208,3351394963,65.0,1,26030,1229,3.0,4.0,False,"POLYGON ((-159725.727 -150026.596, -159726.574...","[2965703173, 3351394963]",11.7,15.0,True
2,208,2931031901,50.0,4,26031,7149,2.3,3.0,False,"POLYGON ((-160856.345 -149985.186, -160851.517...","[2931031901, 2965703173]",11.0,14.0,True
3,208,3652505547,50.0,5,26031,7161,2.3,3.0,False,"POLYGON ((-160766.508 -149944.807, -160769.587...","[2965703173, 3652505547]",11.0,14.0,True
4,8,4271160002,16.0,24,18079,1729,1.0,1.0,False,"POLYGON ((-160857.304 -150003.011, -160854.490...","[2965703173, 4271160002]",9.7,12.0,True
5,8,250359478,54.0,32,18079,1133,2.3,4.0,False,"POLYGON ((-160759.492 -149997.369, -160759.592...","[250359478, 2965703173]",11.0,15.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95973,284,3537423815,19.0,10,32042,3024,1.0,1.7,False,"POLYGON ((-113014.372 -6198.705, -113014.627 -...","[2537623147, 3537423815]",4.0,4.7,True
95974,284,352391758,17.0,56,32082,3112,1.0,1.0,False,"POLYGON ((-113322.384 -6593.601, -113322.500 -...","[352391758, 2537623147]",4.0,4.0,True
95977,284,2070789595,14.0,3,32045,3139,1.0,1.0,False,"POLYGON ((-113857.104 -6692.799, -113854.518 -...","[239624145, 2070789595]",6.0,6.0,True
96088,284,1216220483,33.0,4,32059,3070,3.0,2.7,False,"POLYGON ((-113417.954 -10328.530, -113418.412 ...","[1216220483, 3592067296]",4.3,4.4,True


* drop n_trips from final

### Bay Area Map Image

![map](img/bay.png)

### Los Angeles Map Image

![map](img/la.png)

## Result

* Tested on several operators, provides an approximation of high-quality transit corridors but still incomplete
    * Segments not containing stops will not appear as HQTA-- may need to interpolate (e.g., freeway segments)
    * Various questionable short segments
    * HQTA classification questionable for some routes, will investigate
* Working on improving/documenting code, and towards a statewide proof of concept
* Algorithm is overestimating for SFMTA

## Sandbox

shapes, trips, stop_times, stops

In [30]:
def simple_map(gdf, mouseover=None):
    if 'calitp_extracted_at' in gdf.columns:
        gdf = gdf.drop(columns='calitp_extracted_at')
        gdf.geometry = gdf.geometry.buffer(50)
    
    x = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.x
    y = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)

    if mouseover:
        html = HTML(f'hover to see {mouseover}')
        html.layout.margin = '0px 20px 20px 20px'
        control = WidgetControl(widget=html, position='topright')
        m.add_control(control)

        def update_html(feature,  **kwargs):
            html.value = '''
                <h3><b>{}</b></h3>
            '''.format(feature['properties'][mouseover])
    if 'hq_transit_corr' in gdf.columns:
        geo_data_hq = GeoData(geo_dataframe = gdf[gdf['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#08589e',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'HQTA')

        geo_data_not_hq = GeoData(geo_dataframe = gdf[~gdf['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#a8ddb5',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'non-HQTA')

        m.add_layer(geo_data_hq)
        m.add_layer(geo_data_not_hq)
    
    else:
    
        geo_data_hq = GeoData(geo_dataframe = gdf.to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#08589e',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'gdf')
        m.add_layer(geo_data_hq)
    
    if mouseover:
        geo_data_hq.on_hover(update_html)

    m.add_control(LayersControl())

    return m

### Data Issues

* 61 County Connection has a lot of na departure times?
    * strange results after dropna
* 48 B-Line similarly choppy
* 116 Fresno Area Express has whitespace in departure times
    * ValueError: time data ' 7:04:00' does not match format '%H:%M:%S'

